# VCノードの起動、削除

このNotebookではVCP SDKを利用してVCノードを起動、削除する手順を記しています。

## 概要

![構成](images/101-001.png)

### 用語説明

上の図、またはこれ以降に示す図に記されている構成要素についての簡単な説明を以下に記します。

* [VPC](https://aws.amazon.com/jp/vpc/)
  - Amazon が提供している AWS 内の仮想プライベートネットワーク環境
* [Amazon EC2](https://aws.amazon.com/jp/ec2/)
  - Amazon が提供している仮想コンピューティング環境
* [Azure Virtual Network](https://azure.microsoft.com/ja-jp/services/virtual-network/)
  - Microsoft が提供しているクラウド内の仮想プライベートネットワーク環境
* [Azure Virtual Machine](https://azure.microsoft.com/ja-jp/services/virtual-machines/)
  - Microsoft が提供している仮想コンピューティング環境
* VC Controller
  - VCPがユーザに対して提供しているサービス群
* VC Controller Core
  - ユーザからのAPI呼び出しを受けとりUnitGroupの作成、削除などを行う
* Grafana
  - 可視化ツール、ダッシュボードツール
  - VCP では VCノード のモニタリング状況を表示するのに利用している
* VCP SDK
  - VCPの機能を呼び出して VM/BM の作成、削除などを行うPython3のライブラリ
* VCノード
  - Amazon EC2, Azure Virtual Machine などの計算資源を抽象化したVCPのノード
* Unit
  - 同質（同じ計算資源(cpu, memory, ...)、同じクラウド、同じ用途、...）であるVCノードをまとめて扱うためのもの
* UnitGroup
  - 複数のUnitをまとめて扱うためのものです

## 操作手順

大まかな操作手順は以下のようになります。

1. VCP SDKを用いてVCノード(Amazon EC2 インスタンス)を起動する
1. 起動したVCノードに ssh でログインして操作する
1. GrafanaでVCノードの利用状況を確認する
1. Unit内のVCノードをスケールアウト、スケールインする
1. 別のパブリッククラウド(Microsoft Azure)のVCノードを起動する
1. 起動したVCノードをすべて削除する

## アクセストークンの入力

VCP SDKを利用するにはVC Controllerのアクセストークンが必要となります。次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [1]:
from getpass import getpass
vcc_access_token = getpass()

········


# VCノード(Amazon EC2インスタンス)の起動

VCP SDKを用いてVCノード(Amazon EC2インスタンス)を起動します。また、起動したVCノードに対してsshでログインして操作を行います。

![EC2の起動](images/101-003.png)

## VCP SDKの初期化

VCP SDKの初期化を行います。

In [2]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

# VCP SDKの初期化
vcp = VcpSDK(
    vcc_access_token,  # VCCのアクセストークン
)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2019-XX-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2019-XX-XX XX:XX:XX,XXX - ERROR - 2019/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

この場合はアクセストークンの入力からやり直してください。

`vcp`からは UnitGroup の管理（作成、情報取得）や VCP SDK自体に関する情報取得を行うことができます。例えば、次のセルを実行するとUnitGroupの一覧が表示されます。

> まだUnitGroupを作成していないので、ヘッダー以外はなにも表示されないはずです。

In [3]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate


次のセルを実行すると VCP SDK と VC Controller のバージョンが表示されます。

In [4]:
vcp.version()


vcplib:
  filename: /home/jovyan/vcpsdk/vcplib/occtr.py
  version: 20.10.0+20201001

vcpsdk:
  filename: /home/jovyan/vcpsdk/vcpsdk/vcpsdk.py
  version: 21.04.0+20210401

  plugin:
    aws: 1.2+20191001
    aws_disk: 1.0+20190408
    aws_spot: 1.1+20191001
    azure: 1.2+20191001
    vmware: 1.1+20191001
    azure_disk: 1.0+20190408
    sakura: 1.1+20191001
    sakura_disk: 1.0+20190930
    oracle: 1.0+20200331
    oracle_disk: 1.0+20200331
    aic: 1.2+20191001
    abc: 1.3+20190408
    hokudai: 1.1+20191001
    chameleon: 1.0+20200831
    chameleon_ext: 20200831
    gcp: 1.0+20190408
    onpremises: 1.0+20190408

vc_controller:
  host: localhost
  name: vcc511
  wait_timeout_sec: 1000(default 15min)
  vc_controller: 21.04.0+20210401
  vc_controller_git_tag: 21.04.0-117-gd0b1d69-dirty

  plugin:
    aic: 1.2+20210401
    hokudai: 1.1+20210401
    vmware: 1.1+20210401
    sakura: 1.1+20210401
    aws: 1.2+20210401
    azure: 1.2+20210401
    oracle: 1.0+20210401
    abc: 1.3+20210401


## UnitGroupの作成

`handson101`という名前の UnitGroup を作成します。

In [5]:
# UnitGroupの作成
unit_group = vcp.create_ugroup(
    'handson101'                   # UnitGroupの名前
)

作成したUnitGroupの状態を表示してみます。

> UnitGroup の名前が `handson101` であることと、その状態が `RUNNING` であることなどが確認できるはずです。

In [6]:
print(unit_group)

[Vc]
+ type[compute] name[handson101] vcno(4) state[RUNNING] vcid[eea5d99d2f6944adb5f7dd4fc83fdc9a]



UnitGroupの一覧を表示してみます。

In [7]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate
0,4,compute,handson101,RUNNING,eea5d99d...,2021/07/05 06:22:27 UTC


## UnitGroupの状態を確認する

UnitGroupに属している Unit の一覧を表示します。

> まだ、このUnitGroupでは Unit を作成していないのでヘッダー以外は何も表示されません。

In [8]:
# Unitの一覧を DataFrame で表示する
unit_group.df_units()

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state


UnitGroupに属しているVCノードの一覧を表示します。

> まだVCノードが存在していないので何も表示されません。

In [9]:
unit_group.df_nodes()

[]

## VCノードのspecを指定する

Unitを構成するVCノードがどのような設定であるかを指定するためのオブジェクトとしてVCP SDKでは`spec`オブジェクトを用意しています。VCP SDKの利用者は`spec`オブジェクトのプロパティを設定することで Unitに起動するVCノードの設定内容を定義することができます。

`spec`オブジェクトの設定項目の例を以下に示します。

* 仮想マシンのインスタンスタイプ
  - m5.large, c5.large, ...
* 仮想マシンのルートボリュームサイズ(GiB)
* 仮想マシンに割り当てるプライベートIPアドレス
* Unit内に作成するVCノードの数

設定できる項目はクラウドプロバイダ(aws, azure, ...)毎に異なります。

### flavor の内容を確認する

`spec`オブジェクトの全てのパラメータを毎回設定するのは煩雑になるので典型的な構成のパラメータセットを事前に定義しています。事前に定義した`spec`パラメータセットのことを VCP SDKでは `flavor`と呼んでいます。`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`の定義もクラウドプロバイダ毎に行っています。

次のセルを実行すると aws用に定義している `flavor` の一覧が表示されます。

In [10]:
vcp.df_flavors('aws')

,flavor,instance_type,volume_type,volume_size
0,small,m4.large,gp2,8
1,medium,m4.xlarge,gp2,40
2,large,m4.2xlarge,gp2,100
3,gpu,g3.4xlarge,gp2,40


### spec オブジェクトを作成する

specオブジェクトを作成します。specオブジェクトを作成するにはプロバイダと`flavor`を指定します。ここでは以下の値を指定します。

* プロバイダ: `aws`
* flavor: `small`

In [11]:
spec = vcp.get_spec('aws', 'small')

作成した`spec`の設定内容を確認してみます。

> `instance_type`パラメータに `flavor` の指定と対応するEC2インスタンスタイプ`m4.large`が設定されていることが確認できるはずです。
> `volume_size`, `volume_type`についてもそれぞれ`flavor`と対応する値が設定されます。

In [12]:
print(spec)


aws
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': []}

    instance_type: m4.large
    volume_size: 8
    volume_type: gp2
    volume_id: []
    cloud_image: default
    ip_address_list: []
    tags: {}


`spec`に対しては `flavor`で指定するだけではなく個々のパラメータを直接指定することもできます。例えば、以下のようなものが指定できます。

* num_nodes
  - Unit内に作成するVCノードの数: デフォルト=1
* ip_addresses
  - VCノードに割り当てるプライベートIPアドレスのリスト
  - このパラメータを指定しない場合は利用可能なアドレスが自動的に割り当てられる
* instance_type
  - Amazon EC2のインスタンスタイプ
  - flavorで設定されているもの以外を利用したい場合に指定する
* volume_size
  - Amazon EC2インスタンスのルートボリュームに割り当てるサイズ(GiB)
  - flavorで設定されているもの以外を利用したい場合に指定する
* volume_type
  - Amazon EC2インスタンスのルートボリュームのEBSタイプ
  - flavorで設定されているもの以外を利用したい場合に指定する

試しに `volume_size`を指定してみます。

In [13]:
spec.volume_size = 15

指定した値が`spec`の設定に反映されていることを確認してみます。

In [14]:
print(spec)


aws
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': []}

    instance_type: m4.large
    volume_size: 15
    volume_type: gp2
    volume_id: []
    cloud_image: default
    ip_address_list: []
    tags: {}


`volume_size`の値が指定した値に変更されています。

### sshの鍵ファイルを設定する

VCノードにsshでログインするためには事前に公開鍵認証の鍵を登録する必要があります。そのための設定をここで行います。

VCノードに登録する公開鍵認証の**公開鍵**のパスを次のセルで指定してください。

In [15]:
import os
ssh_public_key = os.path.expanduser('~/.ssh/id_rsa.pub')

指定した公開鍵を `spec` に設定します。

In [16]:
spec.set_ssh_pubkey(ssh_public_key)

sshの公開鍵に関する設定が`spec`に反映されたことを確認してみます。次のセルを実行すると`params`の`e`の`AUTHORIZED_KEYS`に値が設定されていることが確認できます。

In [17]:
print(spec)


aws
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFDc09nTzdmTXJMSG5yRkpSZFROVkVOaWRDNlJTYlNYSE94dmxzSWUvMWc3Nk9IdXB5TE9EUUJ5aXhHc2t1QlVkaXBPaElGV3doeUR4RFRVMU9Xb1BLbHhuTndiV29oZXh1T2s3bmtCYmJ6Y09hT2hhZW5EL1RVdGVLeU05Tm85SW5PTktyS3lTQ0pQVklCUklTVlNXenF6ZzM4NENzSkNNVVBXdVNRcnZmNFpXajI3TDZCYUdaa0c2elVYaVVmMzAvZSsvcitrV2NIZFNFUUcyUHhqRk1iWU9aQkZSMzR5K1RuaWlpTE5LNFlXODgvajdFbXBCRGZ2VEtzL0ZLTXd3WjRjWkdqbmx6eWxFOEdldGhVNC9zcWFYQ1dZSWtHOFJzSjQ2MzRLNU5uTkdRSG80WEZPRzQ0TEJFdkhRTlJoajZCSU14eGd5TkgxVFZlUTdFT2RZNG4gdXNlcjUxMXhlNWIwNTJAMGVhNDIwMGNjMDUxCg==']}

    instance_type: m4.large
    volume_size: 15
    volume_type: gp2
    volume_id: []
    cloud_image: default
    ip_address_list: []
    tags: {}


後でVCノードにログインする際にsshの秘密鍵も必要になるので、ここで設定しておきます。次のセルで**秘密鍵**のパスを指定してください。

In [18]:
ssh_private_key = os.path.expanduser('~/.ssh/id_rsa')

公開鍵と秘密鍵が正しいペアであることをチェックします。次のセルを実行してエラーにならないことを確認してください。

In [19]:
!grep -q "$(ssh-keygen -y -f {ssh_private_key})" {ssh_public_key}

<!---
他にどのようなパラメータが設定できるのかについては
「[VCPSDK APIドキュメント](/handson/files/DOC/index.html#module-vcpsdk.plugins.spec)」を参照してください。
--->

## Unitの作成とVCノードの起動

Unitを作成します。Unitを作成すると同時に VCノード（ここでは Amazon EC2インスタンス）が起動します。

> 処理が完了するまで1分半～2分程度かかります。

In [20]:
# Unitの作成（同時に VCノードが作成される）
unit = unit_group.create_unit(
    'aws-server',               # Unit名の指定
    spec
)

2021-07-05 15:23:12,361 - INFO - BOOTING ... 0 sec
2021-07-05 15:23:17,493 - INFO - BOOTING ... 5 sec
2021-07-05 15:23:22,642 - INFO - BOOTING ... 10 sec
2021-07-05 15:23:27,794 - INFO - BOOTING ... 15 sec
2021-07-05 15:23:32,951 - INFO - BOOTING ... 20 sec
2021-07-05 15:23:38,079 - INFO - BOOTING ... 25 sec
2021-07-05 15:23:43,274 - INFO - BOOTING ... 30 sec
2021-07-05 15:23:48,441 - INFO - BOOTING ... 35 sec
2021-07-05 15:23:53,594 - INFO - BOOTING ... 40 sec
2021-07-05 15:23:58,740 - INFO - BOOTING ... 45 sec
2021-07-05 15:24:03,906 - INFO - BOOTING ... 50 sec
2021-07-05 15:24:09,056 - INFO - BOOTING ... 55 sec
2021-07-05 15:24:14,211 - INFO - BOOTING ... 60 sec
2021-07-05 15:24:19,359 - INFO - BOOTING ... 65 sec
2021-07-05 15:24:24,511 - INFO - BOOTING ... 70 sec
2021-07-05 15:24:29,659 - INFO - BOOTING ... 75 sec
2021-07-05 15:24:35,206 - INFO - BOOTING ... 80 sec
2021-07-05 15:24:40,700 - INFO - BOOTING ... 85 sec
2021-07-05 15:24:45,910 - INFO - BOOTING ... 90 sec
2021-07-05 15:

UnitGroupに属しているUnitの一覧表示を行い、Unitが作成されていることを確認します。

In [21]:
# Unitの一覧を DataFrame で表示する
unit_group.df_units()

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,4,handson101,RUNNING,eea5d99d...,2021/07/05 06:22:27 UTC,aws-server,RUNNING


UnitGroupに属しているVCノードの一覧表示を行い、VCノードが起動していることを確認します。VCノードが正常に起動していることは `node_state` の表示が `RUNNING`になっていることで確認できます。

In [22]:
# VCノードの一覧を DataFrame で表示する
unit_group.df_nodes()

,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,4,handson101,aws-server,RUNNING,1,5ebea5e8...,RUNNING,172.30.2.188,i-0598cb2b417048269,VCP-8dbe841c-eea5d99d,none


ここでは VCノードの起動が完了するまで待ち合わせるモードでUnitの作成を行いましたが、非同期処理でUnit, VCノードの作成を行うこともできます。`create_unit()`の`wait_for`パラメータに`False`を指定すると非同期モードでのUnit, VCノードの作成が行えます。

```
unit = unit_group.create_unit('aws-server', spec, wait_for=False)
```

非同期モードでUnitを作成した場合にUnitやVCノードが起動したことを確認するには、以下に示す方法で状態取得を行ってください。

1. `unit_group.df_units()`
  * Unitの一覧表示の`unit_state`の項目で Unitの状態を確認できます
  * 起動中は `BOOTING`、起動が完了して実行中になると `RUNNING` と表示されます
 
1. `unit.df_nodes()`, `unit_group.df_nodes()`
  * VCノードの一覧表示の`node_state`の項目で VCノードの状態を確認できます
  * 起動中は `BOOTING`、起動が完了して実行中になると `RUNNING` と表示されます

非同期モードでUnit作成を行った場合でも、以下のメソッドを利用することで後からVCノード起動の待ち合わせを行うことができます。
* `unit_group.wait_unit_applied(unit_name)`
  - unit_nameで指定されたUnitの状態が `RUNNING` または `ERROR`になるのを待ちます

## 疎通確認

起動した VCノードに対して`ping`を行ってみます。

まずUnitGroup内で起動しているVCノードに割り当てられているプライベートIPアドレスの値を取得して変数 `ip_address`に格納します。

In [23]:
# unit_group.find_ip_addresses() は UnitGroup内の全VCノードのIPアドレスのリストを返します
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')[0] # 今は１つのVCノードのみ起動しているので [0] で最初の要素を取り出す

print(ip_address)

172.30.2.188


実際に `ping` を行ってみます。

> Codeセルで先頭に `!` をつけるとシェルコマンドが実行できます。また `{}`で囲むことで Python の変数参照やコードの実行に置き換えることができます。詳しくは「[IPython Documentation](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#system-shell-commands)」などを参照してください。

In [24]:
# NIIのハンズオン環境ではネットワーク構成の事情により本NotebookからVCノードへのping実行はできません。
# !ping -c 5 {ip_address}

# VCノードに ssh でログインして操作する

起動したVCノードに ssh でログインして操作を行います。

![](./images/101-006.png)

まず、ログインする前に ~/.ssh/known_hosts のホストキーを更新します。

In [25]:
!touch ~/.ssh/known_hosts
# ~/.ssh/known_hosts から古いホストキーを削除する
!ssh-keygen -R {ip_address}

# ホストキーの登録
!ssh-keyscan -H {ip_address} >> ~/.ssh/known_hosts

Host 172.30.2.188 not found in /home/jovyan/.ssh/known_hosts


`ls -la` を実行してみます。

In [26]:
!ssh -i {ssh_private_key} -l root {ip_address} /bin/ls -la

total 20
drwx------    1 root     root          4096 Oct  5  2020 .
drwxr-xr-x    1 root     root          4096 Jul  5 06:25 ..
drwxr-xr-x    1 root     root          4096 Oct  5  2020 .cache
drwxr-xr-x    3 root     root          4096 Oct  5  2020 .gem
drwx------    1 root     root          4096 Jul  5 06:25 .ssh


VCノードにsshでログインするにはsshの引数に以下の指定が必要となります。

* sshの identity ファイル(-i)
  - `spec`オブジェクトに設定した公開鍵に対応する秘密鍵
* ユーザ名(-l)
  - `root`
* ログイン先のIPアドレス

sshのオプションを毎回指定するのも煩雑なので変数に設定しておきます。

In [27]:
ssh_opts = f"-i {ssh_private_key} -l root"

VCノードに対して`uname -a`, `df -h` などを実行してみます。

In [28]:
!ssh {ssh_opts} {ip_address} uname -a

Linux ip-172-30-2-188 4.4.0-159-generic #187-Ubuntu SMP Thu Aug 1 16:28:06 UTC 2019 x86_64 Linux


In [29]:
!ssh {ssh_opts} {ip_address} df -h

Filesystem                Size      Used Available Use% Mounted on
overlay                  14.6G      3.3G     10.6G  24% /
tmpfs                     3.9G         0      3.9G   0% /sys/fs/cgroup
udev                      3.9G         0      3.9G   0% /dev
tmpfs                     3.9G         0      3.9G   0% /dev/shm
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/resolv.conf
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/hostname
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/hosts
/dev/xvda1               14.6G      3.3G     10.6G  24% /var/lib/docker
none                      3.9G         0      3.9G   0% /tmp


# GrafanaでVCノードの利用状況を確認する

VCPではVCノードの利用状況(CPU負荷、メモリ使用量、ネットワーク）を確認するためGrafanaのダッシュボードを提供しています。

## Grafanaへのログイン

以下のセルを実行して Grafana ダッシュボードを開いてください。

最初にログイン画面が表示されるので ユーザ名、パスワードにそれぞれ `admin`, `admin`を入力してください。 

In [30]:
vcc_ctr = vcp.vcc_info()['host']
http_host = vcc_ctr.split(':')[0]
grafana_url = "https://{}/grafana/d/vcp/vcp-metrics?refresh=5s".format(http_host)
print(grafana_url)

https://localhost/grafana/d/vcp/vcp-metrics?refresh=5s


![Grafanaのログイン画面](./images/grafana-login.png)

## 負荷状況のグラフ表示

以下のように表示されます。左半分のグラフがBaseコンテナの負荷状況で、右半分がアプリケーションコンテナの負荷状況のグラフです。

![負荷状況](./images/grafana-metrics.png)

VCノードに負荷をかけるために、アプリケーションコンテナで stress コマンドを実行します。

In [31]:
# 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
!ssh {ssh_opts} {ip_address} \
    /usr/local/bin/docker run -td --rm --name stress-0 polinux/stress \
    stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
63bc94deeb28: Pulling fs layer
8d43371bad93: Pulling fs layer
63bc94deeb28: Verifying Checksum
63bc94deeb28: Download complete
8d43371bad93: Verifying Checksum
8d43371bad93: Download complete
63bc94deeb28: Pull complete
8d43371bad93: Pull complete
Digest: sha256:b6144f84f9c15dac80deb48d3a646b55c7043ab1d83ea0a697c09097aaad21aa
Status: Downloaded newer image for polinux/stress:latest
9bf8a3e959bf02dafd8237ec769bb3205b20abcd03b875666a5e51e918be23b3


In [32]:
# 60秒間だけ CPU x 2 とメモリ 128 MB を消費する (2回目)
!ssh {ssh_opts} {ip_address} \
    /usr/local/bin/docker run -td --rm --name stress-1 polinux/stress \
    stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

f93c5d21d358bcb9009640687827dca94aa5a3de159be4a48f42aaf10b8f7902


# VCノードのスケールアウト、スケールイン

![スケールアウト、スケールイン](images/101-004.png)

## VCノードのスケールアウト

Unitには `spec`オブジェクトに指定されている設定と同一の VCノードを作成する機能があります。その機能を利用して Unit にVCノードを追加してみます。

まず、現在のVCノードの状況を確認しておきます。

In [ ]:
# VCノードの一覧を DataFrame で表示する
unit.df_nodes()

Unit に VCノードを追加するには`unit`オブジェクトに対して `add_nodes()` を呼び出します。次のセルを実行するとUnitに VCノード が追加されます。

> VCノード の起動には 1分半～2分程度かかります。

In [ ]:
unit.add_nodes()

`add_node()`を呼び出すことで追加されるVCノードの数はデフォルトでは１ですが、パラメータ`num_add_nodes`を設定することで追加するVCノード数を変更することが出来ます。例えば３ノードを追加するには以下のような指定を行います。

```
unit.add_nodes(num_add_nodes=3)
```
また、追加するVCノードのIPアドレスを指定する場合は、以下のような指定を行います。
```
unit.add_nodes(ip_address='172.30.2.30')
```
他にどんなパラメータを指定することが出来るかについては次のセルのように Codeセルの先頭で `?` を指定することで確認できます。

In [ ]:
?unit.add_nodes

追加したVCノードの状態を確認するために、VCノードの一覧表示をおこなってみます。

In [ ]:
unit.df_nodes()

起動している全てのVCノードに対して `ping` を行ってみます。

In [ ]:
# Unit内のVCノードのIPアドレスのリストを取得する
ip_address = unit.find_ip_addresses(node_state='RUNNING')
print(ip_address)

# それぞれのIPアドレスに対して pingコマンドを実行する
for address in ip_address:
    !ping -c 5 {address}
    print()

VCノードを追加したので ~/.ssh/known_hosts を更新します。

In [ ]:
for address in ip_address:
    # ~/.ssh/known_hosts から古いホストキーを削除する
    !ssh-keygen -R {address}
    # ホストキーの登録
    !ssh-keyscan -H {address} >> ~/.ssh/known_hosts

追加で起動した VCノード は同じ Unit に属しているため同じ構成(cpu, memory, disk)になっています。そのことを確認するために、sshでログインして以下のコマンドを実行してみます。

* カーネルバージョン
  - uname -a
* CPU
  - cat /proc/cpuinfo
* メモリ量
  - free
* ディスク容量
  - df -h
  
> ただし aws では同じインスタンスタイプに複数種類の CPU が混在していることがあり CPU が微妙に異なることがあります。

In [ ]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} uname -a
    print()

In [ ]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} cat /proc/cpuinfo | grep 'model name'
    print()

In [ ]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} free
    print()

In [ ]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} df -h
    print()

先程と同様に、負荷をかけるためにそれぞれのVCノードでアプリケーションコンテナによる stress コマンドの実行を行います。

In [ ]:
for idx, address in enumerate(ip_address):
    # 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
    !ssh {ssh_opts} {address} \
        /usr/local/bin/docker run -td --rm --name stress-{idx} polinux/stress \
        stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

[Grafana](/grafana/d/vcp/vcp-metrics?refresh=5s) で利用状況を確認してみます。

## VCノードのスケールイン

現在Unitには２つのVCノードがありますが、そのうちの一つを削除してみます。

まず、現在のVCノードの状況を確認しておきます。

In [ ]:
# VCノードの一覧を DataFrame で表示する
unit.df_nodes()

Unit から VCノードを削除するにはオブジェクト`unit`に対して `delete_nodes()` を呼び出します。

複数の VCノード のうちどのVCノード を削除するかを引数で指定することができます。ここでは`ip_address`パラメータを指定してVCノードに割り当てられている IPアドレスで VCノードを特定します。

> VCノードを特定するパラメータが指定されなかった場合は削除対象となる VCノードは VCP SDK によって自動的に選択されます。

VCノードを特定するための他のパラメータに指定方法については次のセルを実行すると表示される情報で確認してください。

In [ ]:
?unit.delete_nodes

では実際にVCノードを削除します。まず削除対象とするVCノードのIPアドレスを確認します。スケールアウトの節で追加したVCノードのIPアドレスを指定することにします。

In [ ]:
# 削除対象とするVCノードの IPアドレスを表示 
print(ip_address[-1])                         # 後から追加した VCノード を削除対象とする

次のセルを実行するとUnit からVCノードが削除され、対応するAWS EC2インスタンスも削除されます。

> VCノード の削除には 1分～2分程度かかります。

In [ ]:
# VCノードの削除
unit.delete_nodes(
    ip_address=ip_address[-1]     # IPアドレスで削除対象を指定する
)

Unit内のVCノードの一覧を確認してみます。VCノードが減っていることが確認できるはずです。

In [ ]:
unit.df_nodes()

先ほど取得した２つのVCノードのIPアドレスに対して `ping` を行ってみます。
削除されている VCノード に対する `ping` が失敗するので、次のセルは必ずエラーになるはずです。

In [ ]:
for address in ip_address:
    !ping -c 5 {address}

# 別のクラウドでVCノードを起動する

![azure](images/101-005.png)

## Unit の追加

先程とは異なる構成のVCノードを追加するので、新たな Unit を作成します。

### VCノードのspecを指定する

AWSの場合と同様 `spec` オブジェクトを作成します。

AWSの場合からの変更点は、specの作成等で指定するプロバイダ名を、使用するプロバイダのものに変更するだけです。
ここでの具体的な変更箇所は次の2つのAPIの引数になります。

- `VCPSDK::df_flavors()`
- `VCPSDK::get_spec()`

VCP SDKで現在使用可能なクラウドプロバイダと、それを使用する際の引数に指定するプロバイダ名の値は以下のとおりです。

|  クラウドプロバイダ  |  プロバイダ名  |
| :---- | :---- |
|  Amazon EC2  |  `aws`  |
|  Microsoft Azure  |  `azure`  |
|  Oracle Cloud Infrastructure  |  `oracle`  |
|  さくらのクラウド  |  `sakura` |
|  VMware vSphere  |  `vmware`  |

ここでは、Microsoft Azure でVCノードを作成します。
下記の `'azure'` を指定した箇所を変更することで、他のプロバイダでも同様の操作をすることができます。

まず最初に、Microsoft Azure 用に定義している `flavor` の内容を確認します。

In [33]:
vcp.df_flavors('azure')

,flavor,vm_size,disk_size_gb,managed_disk_type
0,small,Standard_B1s,40,Standard_LRS
1,medium,Standard_D3_v2,100,Standard_LRS
2,large,Standard_D4_v2,100,Premium_LRS


続いて、以下のパラメータを指定して `spec`オブジェクトを作成します。
* プロバイダ: `azure`
* flavor: `medium`

In [34]:
spec_azure = vcp.get_spec('azure', 'medium')

作成した`spec_azure`の設定内容を確認してみます。

In [35]:
print(spec_azure)


azure
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': []}


        vm_size: Standard_D3_v2
        disk_size_gb: 100
        managed_disk_type: Standard_LRS
        ext_managed_disk: []
        cloud_image: default
        ip_address_list: []
    tags: 
        tags: {}


指定した `flavor`に対応する `vm_size`, `disk_size_gb` などが設定されていることが確認できます。

### sshの公開鍵設定

VCノードに登録するsshの公開鍵を `spec_azure`に設定します。先ほど aws 向けの`spec`に設定したのと同じ値を指定しています。

In [36]:
spec_azure.set_ssh_pubkey(ssh_public_key)

### specの設定内容を確認する

`spec_azure`の設定内容を確認してみます。

In [37]:
print(spec_azure)


azure
------------------------

    cci_version: "1.1"
    name: 
    description: 
    private_network: default
    num_nodes: 1
    image: vcp/base:1.6.2
    recovery_mode: none
    docker_opt: {}
    params:
        {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFDc09nTzdmTXJMSG5yRkpSZFROVkVOaWRDNlJTYlNYSE94dmxzSWUvMWc3Nk9IdXB5TE9EUUJ5aXhHc2t1QlVkaXBPaElGV3doeUR4RFRVMU9Xb1BLbHhuTndiV29oZXh1T2s3bmtCYmJ6Y09hT2hhZW5EL1RVdGVLeU05Tm85SW5PTktyS3lTQ0pQVklCUklTVlNXenF6ZzM4NENzSkNNVVBXdVNRcnZmNFpXajI3TDZCYUdaa0c2elVYaVVmMzAvZSsvcitrV2NIZFNFUUcyUHhqRk1iWU9aQkZSMzR5K1RuaWlpTE5LNFlXODgvajdFbXBCRGZ2VEtzL0ZLTXd3WjRjWkdqbmx6eWxFOEdldGhVNC9zcWFYQ1dZSWtHOFJzSjQ2MzRLNU5uTkdRSG80WEZPRzQ0TEJFdkhRTlJoajZCSU14eGd5TkgxVFZlUTdFT2RZNG4gdXNlcjUxMXhlNWIwNTJAMGVhNDIwMGNjMDUxCg==']}


        vm_size: Standard_D3_v2
        disk_size_gb: 100
        managed_disk_type: Standard_LRS
        ext_managed_disk: []
        cloud_image: default
        ip_address_list: []
    tags:

aws向けに最初に作成した `spec`オブジェクトと`spec_azure`との差分を確認してみます。

> `spec`オブジェクトの内部では aws は `instance_type`、azure は`vm_size` などそれぞれのクラウドプロバイダに応じたパラメータが設定されていることが確認できます。

In [38]:
from difflib import unified_diff
import sys

sys.stdout.writelines(unified_diff(
    str(spec).splitlines(True),
    str(spec_azure).splitlines(True),
    fromfile='aws',
    tofile='azure',
))

--- aws
+++ azure
@@ -1,10 +1,10 @@
 
-aws
+azure
 ------------------------
 
     cci_version: "1.1"
-    name: aws-server
+    name: 
     description: 
     private_network: default
     num_nodes: 1
@@ -14,11 +14,13 @@
     params:
         {'v': [], 'e': ['AUTHORIZED_KEYS=c3NoLXJzYSBBQUFBQjNOemFDMXljMkVBQUFBREFRQUJBQUFCQVFDc09nTzdmTXJMSG5yRkpSZFROVkVOaWRDNlJTYlNYSE94dmxzSWUvMWc3Nk9IdXB5TE9EUUJ5aXhHc2t1QlVkaXBPaElGV3doeUR4RFRVMU9Xb1BLbHhuTndiV29oZXh1T2s3bmtCYmJ6Y09hT2hhZW5EL1RVdGVLeU05Tm85SW5PTktyS3lTQ0pQVklCUklTVlNXenF6ZzM4NENzSkNNVVBXdVNRcnZmNFpXajI3TDZCYUdaa0c2elVYaVVmMzAvZSsvcitrV2NIZFNFUUcyUHhqRk1iWU9aQkZSMzR5K1RuaWlpTE5LNFlXODgvajdFbXBCRGZ2VEtzL0ZLTXd3WjRjWkdqbmx6eWxFOEdldGhVNC9zcWFYQ1dZSWtHOFJzSjQ2MzRLNU5uTkdRSG80WEZPRzQ0TEJFdkhRTlJoajZCSU14eGd5TkgxVFZlUTdFT2RZNG4gdXNlcjUxMXhlNWIwNTJAMGVhNDIwMGNjMDUxCg==']}
 
-    instance_type: m4.large
-    volume_size: 15
-    volume_type: gp2
-    volume_id: []
-    cloud_image: default
-    ip_address_list: []
-    tags: {}
+
+        v

### Unitの作成とVCノードの起動

まず作成前のUnitGroup状態を確認しておきます。

In [39]:
from IPython.display import display

# Unitの一覧を DataFrame で表示する
display(unit_group.df_units())

# VCノードの一覧を DataFrame で表示する
display(unit_group.df_nodes())

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,4,handson101,RUNNING,eea5d99d...,2021/07/05 06:22:27 UTC,aws-server,RUNNING


,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,4,handson101,aws-server,RUNNING,1,5ebea5e8...,RUNNING,172.30.2.188,i-0598cb2b417048269,VCP-8dbe841c-eea5d99d,none


Unitを作成します。

> 処理が完了するまでには 2～5分程度かかります。

In [40]:
# Unitの作成（同時に VCノードが作成される）
unit2 = unit_group.create_unit(
            'azure-server',    # Unit 名
            spec_azure         # spec オブジェクト
        )

2021-07-05 15:27:32,166 - INFO - BOOTING ... 0 sec
2021-07-05 15:27:37,397 - INFO - BOOTING ... 5 sec
2021-07-05 15:27:42,661 - INFO - BOOTING ... 10 sec
2021-07-05 15:27:47,884 - INFO - BOOTING ... 15 sec
2021-07-05 15:27:53,073 - INFO - BOOTING ... 20 sec
2021-07-05 15:27:58,297 - INFO - BOOTING ... 25 sec
2021-07-05 15:28:03,499 - INFO - BOOTING ... 30 sec
2021-07-05 15:28:08,724 - INFO - BOOTING ... 35 sec
2021-07-05 15:28:13,961 - INFO - BOOTING ... 40 sec
2021-07-05 15:28:19,186 - INFO - BOOTING ... 45 sec
2021-07-05 15:28:24,414 - INFO - BOOTING ... 50 sec
2021-07-05 15:28:29,614 - INFO - BOOTING ... 55 sec
2021-07-05 15:28:34,791 - INFO - BOOTING ... 60 sec
2021-07-05 15:28:40,031 - INFO - BOOTING ... 65 sec
2021-07-05 15:28:45,239 - INFO - BOOTING ... 70 sec
2021-07-05 15:28:50,477 - INFO - BOOTING ... 75 sec
2021-07-05 15:28:55,679 - INFO - BOOTING ... 80 sec
2021-07-05 15:29:00,952 - INFO - BOOTING ... 85 sec
2021-07-05 15:29:06,236 - INFO - BOOTING ... 90 sec
2021-07-05 15:

Unit作成後の状態を確認します。Unit とVCノードがそれぞれ２つあることが確認できるはずです。VCノード一覧ではそれぞれのVCノードの unit_name が異なることが確認できるはずです。またawsとazureのVCノードには**異なるサブネット**のIPアドレスが割り当てられていることが確認できます。

> 各プロバイダにどのようなサブネットを割り当てているのかについては[VCP Manager Web UI](https://vcpdev.ecloud.nii.ac.jp/vcp/v1/ui/occmgr/menu/occtrs) で確認ができます。

In [41]:
# Unitの一覧を DataFrame で表示する
display(unit_group.df_units())

# VCノードの一覧を DataFrame で表示する
display(unit_group.df_nodes())

,vctype,vcno,vcname,vc_state,vcid,cdate,unit_name,unit_state
0,compute,4,handson101,RUNNING,eea5d99d...,2021/07/05 06:22:27 UTC,aws-server,RUNNING
1,compute,4,handson101,RUNNING,eea5d99d...,2021/07/05 06:22:27 UTC,azure-server,RUNNING


,vcno,vcname,unit_name,unit_state,node_no,node_id,node_state,cloud_instance_address,cloud_instance_id,cloud_instance_name,volumes
0,4,handson101,aws-server,RUNNING,1,5ebea5e8...,RUNNING,172.30.2.188,i-0598cb2b417048269,VCP-8dbe841c-eea5d99d,none
1,4,handson101,azure-server,RUNNING,1,8b011e6e...,RUNNING,172.20.2.5,/subscriptions/d47388de-3ed7-4aa6-9e7c-5c09ecf55fd8/resourceGroups/niivcp/providers/Microsoft.Compute/virtualMachines/8b011e6evm,VCP-8dbe841c-eea5d99d,none


VCノードを追加したので ~/.ssh/known_hosts を更新します。

In [42]:
for addr in unit_group.find_ip_addresses(node_state='RUNNING'):
    !ssh-keygen -R {addr}
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

# Host 172.30.2.188 found: line 3
/home/jovyan/.ssh/known_hosts updated.
Original contents retained as /home/jovyan/.ssh/known_hosts.old
Host 172.20.2.5 not found in /home/jovyan/.ssh/known_hosts


### 疎通確認

VCノードに対して`ping` を実行してみます。

![](./images/101-007.png)

In [43]:
# UnitGroup内にあるVCノードのIPアドレスリストを取得する
ip_address = unit_group.find_ip_addresses(node_state='RUNNING')

# それぞれのVCノードに対してpingを実行する
# NIIのハンズオン環境ではネットワーク構成の事情により本NotebookからVCノードへのping実行はできません。
# for address in ip_address:
#     !ping -c 5 {address}
#     print()

VCPでは複数の異なるクラウドを仮想的に一つの計算基盤（仮想クラウド）のように扱える機能を提供しています。そのため異なるクラウド(aws, azure)で起動したVCノードであっても、追加の設定なしに互いに通信することができます。そのことを確認するために、作成した２つの VCノード の一方から他方に対して `ping` を行ってみます。

In [44]:
# pingを実行するVCノードのIPアドレスを表示する
print(ip_address[0])
print()

!ssh {ssh_opts} {ip_address[0]} \
    ping -c 10 {ip_address[1]}

172.30.2.188

PING 172.20.2.5 (172.20.2.5): 56 data bytes
64 bytes from 172.20.2.5: seq=0 ttl=62 time=6.551 ms
64 bytes from 172.20.2.5: seq=1 ttl=62 time=7.506 ms
64 bytes from 172.20.2.5: seq=2 ttl=62 time=6.934 ms
64 bytes from 172.20.2.5: seq=3 ttl=62 time=6.854 ms
64 bytes from 172.20.2.5: seq=4 ttl=62 time=6.741 ms
64 bytes from 172.20.2.5: seq=5 ttl=62 time=7.263 ms
64 bytes from 172.20.2.5: seq=6 ttl=62 time=7.906 ms
64 bytes from 172.20.2.5: seq=7 ttl=62 time=6.722 ms
64 bytes from 172.20.2.5: seq=8 ttl=62 time=7.369 ms
64 bytes from 172.20.2.5: seq=9 ttl=62 time=6.923 ms

--- 172.20.2.5 ping statistics ---
10 packets transmitted, 10 packets received, 0% packet loss
round-trip min/avg/max = 6.551/7.076/7.906 ms


逆方向で同様のことを行ってみます。

In [45]:
# pingを実行するVCノードのIPアドレスを表示する
print(ip_address[1])
print()

!ssh {ssh_opts} {ip_address[1]} \
    ping -c 10 {ip_address[0]}

172.20.2.5

PING 172.30.2.188 (172.30.2.188): 56 data bytes
64 bytes from 172.30.2.188: seq=0 ttl=62 time=6.895 ms
64 bytes from 172.30.2.188: seq=1 ttl=62 time=6.897 ms
64 bytes from 172.30.2.188: seq=2 ttl=62 time=8.655 ms
64 bytes from 172.30.2.188: seq=3 ttl=62 time=7.383 ms
64 bytes from 172.30.2.188: seq=4 ttl=62 time=6.720 ms
64 bytes from 172.30.2.188: seq=5 ttl=62 time=6.644 ms
64 bytes from 172.30.2.188: seq=6 ttl=62 time=6.865 ms
64 bytes from 172.30.2.188: seq=7 ttl=62 time=7.177 ms
64 bytes from 172.30.2.188: seq=8 ttl=62 time=7.179 ms
64 bytes from 172.30.2.188: seq=9 ttl=62 time=6.645 ms

--- 172.30.2.188 ping statistics ---
10 packets transmitted, 10 packets received, 0% packet loss
round-trip min/avg/max = 6.644/7.106/8.655 ms


## VCノードに ssh でログインして操作する

![](./images/101-008.png)

Azureに追加で起動した VCノード は aws とは異なる構成(cpu, memory, disk)になってるはずです。そのことを確認するために、sshでログインして以下のコマンドを実行してみます。

* カーネルバージョン
  - uname -a
* CPU
  - cat /proc/cpuinfo
* メモリ量
  - free
* ディスク容量
  - df -h

In [46]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} uname -a
    print()

172.30.2.188:
Linux ip-172-30-2-188 4.4.0-159-generic #187-Ubuntu SMP Thu Aug 1 16:28:06 UTC 2019 x86_64 Linux

172.20.2.5:
Linux 8b011e6e 4.15.0-1082-azure #92~16.04.1-Ubuntu SMP Tue Apr 14 22:28:34 UTC 2020 x86_64 Linux



In [47]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} cat /proc/cpuinfo | grep 'model name'
    print()

172.30.2.188:
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz

172.20.2.5:
model name	: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
model name	: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
model name	: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz
model name	: Intel(R) Xeon(R) Platinum 8171M CPU @ 2.60GHz



In [48]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} free
    print()

172.30.2.188:
              total        used        free      shared  buff/cache   available
Mem:        8173364      409908     5910852       12448     1852604     7632552
Swap:             0           0           0

172.20.2.5:
              total        used        free      shared  buff/cache   available
Mem:       14361896      645084    11585708        8868     2131104    13562924
Swap:             0           0           0



In [49]:
for address in ip_address:
    print(address + ':')
    !ssh {ssh_opts} {address} df -h
    print()

172.30.2.188:
Filesystem                Size      Used Available Use% Mounted on
overlay                  14.6G      3.3G     10.6G  24% /
tmpfs                     3.9G         0      3.9G   0% /sys/fs/cgroup
udev                      3.9G         0      3.9G   0% /dev
tmpfs                     3.9G         0      3.9G   0% /dev/shm
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/resolv.conf
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/hostname
/dev/xvda1               14.6G      3.3G     10.6G  24% /etc/hosts
/dev/xvda1               14.6G      3.3G     10.6G  24% /var/lib/docker
none                      3.9G         0      3.9G   0% /tmp

172.20.2.5:
Filesystem                Size      Used Available Use% Mounted on
overlay                  96.7G      2.8G     94.0G   3% /
tmpfs                     6.8G         0      6.8G   0% /sys/fs/cgroup
udev                      6.8G         0      6.8G   0% /dev
tmpfs                     6.8G         0      6.8G 

最後に先程と同様に、負荷をかけるためにそれぞれのVCノードで stress コマンドを実行します。これまで同様 stressコマンドはアプリケーションコンテナとして実行します。

In [50]:
for unit in unit_group.find_units():
    address = unit.find_ip_addresses(node_state='RUNNING')
    # 60秒間だけ CPU x 2 とメモリ 128 MB を消費する
    !ssh {ssh_opts} {address[0]} \
        /usr/local/bin/docker run -td --rm --name stress-{unit.name} polinux/stress \
        stress --cpu 2 --io 1 --vm 2 --vm-bytes 128M --timeout 60s --verbose

3f00d4f6d3f32db5653569a536a60f5e72703055f17028c876f9d00d6ed69e40
Unable to find image 'polinux/stress:latest' locally
latest: Pulling from polinux/stress
63bc94deeb28: Pulling fs layer
8d43371bad93: Pulling fs layer
63bc94deeb28: Verifying Checksum
63bc94deeb28: Download complete
8d43371bad93: Download complete
63bc94deeb28: Pull complete
8d43371bad93: Pull complete
Digest: sha256:b6144f84f9c15dac80deb48d3a646b55c7043ab1d83ea0a697c09097aaad21aa
Status: Downloaded newer image for polinux/stress:latest
c2bb3c8df18e065c4aa6f45cbbebb569458c00a0819f28a9566862fe4fccadc4


Grafanaで利用状況を確認してみます。

In [51]:
print(grafana_url)

https://localhost/grafana/d/vcp/vcp-metrics?refresh=5s


# 全てのVCノードを削除する

ここまで作成した全てのリソース（UnitGroup, Unit、VCノード）を削除します。この操作を行うことで AWS EC2インスタンスやAzure VMなどのクラウドに作成したリソースが削除されます。

> 全てのリソースの削除には 4～5分程度かかります。

In [52]:
unit_group.cleanup()

2021-07-05 15:33:32,070 - INFO - aws-server is DELETING
azure-server is DELETING ... 0 sec
2021-07-05 15:33:37,248 - INFO - aws-server is DELETING
azure-server is DELETING ... 5 sec
2021-07-05 15:33:42,477 - INFO - aws-server is DELETING
azure-server is DELETING ... 10 sec
2021-07-05 15:33:47,793 - INFO - aws-server is DELETING
azure-server is DELETING ... 15 sec
2021-07-05 15:33:52,991 - INFO - aws-server is DELETING
azure-server is DELETING ... 20 sec
2021-07-05 15:33:58,202 - INFO - aws-server is DELETING
azure-server is DELETING ... 25 sec
2021-07-05 15:34:03,395 - INFO - aws-server is DELETING
azure-server is DELETING ... 30 sec
2021-07-05 15:34:08,601 - INFO - aws-server is DELETING
azure-server is DELETING ... 35 sec
2021-07-05 15:34:13,789 - INFO - aws-server is DELETING
azure-server is DELETING ... 40 sec
2021-07-05 15:34:18,944 - INFO - azure-server is DELETING ... 45 sec
2021-07-05 15:34:24,070 - INFO - azure-server is DELETING ... 50 sec
2021-07-05 15:34:29,220 - INFO - azu

削除後の状態を確認してみます。

In [53]:
# UnitGroupの一覧を DataFrame で表示する
vcp.df_ugroups()

,vcno,vctype,vcname,state,vcid,cdate
